In [1]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import pandas as pd
from pprint import pprint
from datetime import datetime
import os
import csv

In [2]:
# Save config information.
url = "https://data.ca.gov/api/3/action/datastore_search?resource_id=d73ee828-c2c6-485c-91dc-c26bd9ce3991&"

# Build partial query URL
limit = 1000
year = 2013
query_url = f"{url}&limit={limit}&q={year}"
response = requests.get(query_url).json()
response

{'help': 'https://data.ca.gov/api/3/action/help_show?name=datastore_search',
 'success': True,
 'result': {'include_total': True,
  'limit': 1000,
  'q': '2013',
  'records_format': 'objects',
  'resource_id': 'd73ee828-c2c6-485c-91dc-c26bd9ce3991',
  'total_estimation_threshold': None,
  'records': [{'_id': 3983,
    'OBJECTID': '3983',
    'Year': '2013',
    'State': 'CA',
    'Agency': 'USF',
    'Unit ID': 'STF',
    'Fire Name': 'RIM',
    'Local Incident Number': '00000006',
    'Alarm Date': '8/17/2013 12:00:00 AM',
    'Containment Date': '10/24/2013 12:00:00 AM',
    'Cause': '4',
    'Collection Method': '7',
    'Management Objective': '1',
    'GIS Calculated Acres': '256175.5',
    'Comments': '2013-CAYNP-0126 Suppressed Human Caused',
    'Complex Name': None,
    'IRWIN ID': None,
    'Fire Number (historical use)': None,
    'Complex ID': None,
    'DECADES': '2010',
    'Shape__Area': '1667948766.79688',
    'Shape__Length': '432168.678063784',
    'rank': 0.0706241},

In [3]:
df = pd.DataFrame(response['result']['records'])
df_2013 = df.loc[df['Year'] == f'{str(year)}']
columns_to_keep = ['Year', 'Alarm Date','Containment Date','Cause','GIS Calculated Acres']
columns_to_drop = [col for col in df_2013.columns if col not in columns_to_keep]
df_2013 = df_2013.drop(columns=columns_to_drop)
df_2013 = df_2013.dropna (subset = ["Containment Date"])
df_2013.head()

,Year,Alarm Date,Containment Date,Cause,GIS Calculated Acres
0,2013,8/17/2013 12:00:00 AM,10/24/2013 12:00:00 AM,4,256175.5
1,2013,9/10/2013 12:00:00 AM,9/16/2013 12:00:00 AM,1,0.07944366
2,2013,5/27/2013 12:00:00 AM,5/28/2013 12:00:00 AM,10,1.235317
3,2013,9/18/2013 12:00:00 AM,9/19/2013 12:00:00 AM,14,3.633029
4,2013,7/8/2013 12:00:00 AM,7/8/2013 12:00:00 AM,2,0.608927


In [4]:
df_2013['Alarm Date'] = [dt.replace(' 12:00:00 AM', '') for dt in df_2013['Alarm Date']]
df_2013['Containment Date'] = [str(item) for item in df_2013['Containment Date']]
df_2013['Containment Date'] = [dt.replace(' 12:00:00 AM', '') for dt in df_2013['Containment Date']]
df_2013.head()

,Year,Alarm Date,Containment Date,Cause,GIS Calculated Acres
0,2013,8/17/2013,10/24/2013,4,256175.5
1,2013,9/10/2013,9/16/2013,1,0.07944366
2,2013,5/27/2013,5/28/2013,10,1.235317
3,2013,9/18/2013,9/19/2013,14,3.633029
4,2013,7/8/2013,7/8/2013,2,0.608927


In [5]:
df_2013 = df_2013.rename(columns={'Cause': 'Cause #'})
df_2013['Cause'] = ''
df_2013['Cause #'] = df_2013['Cause #'].astype(int)
def map_cause_to_meaning(i):
    if i == 1:
        return 'Lightning'
    elif i == 2:
        return 'Equipment Use'
    elif i == 3:
        return 'Smoking'
    elif i == 4:
        return 'Campfire'
    elif i == 5:
        return 'Debris'
    elif i == 6:
        return 'Railroad'
    elif i == 7:
        return 'Arson'
    elif i == 8:
        return 'Playing with fire'
    elif i == 9:
        return 'Miscellaneous'
    elif i == 10:
        return 'Vehicle'
    elif i == 11:
        return 'Powerline'
    elif i == 12:
        return 'Firefighter Training'
    elif i == 13:
        return 'Non-Firefighter Training'
    elif i == 14:
        return 'Unknown / Unidentified'
    elif i == 15:
        return 'Structure'
    elif i == 16:
        return 'Aircraft'
    elif i == 18:
        return 'Escaped Prescribed Burn'
    elif i == 19:
        return 'Illegal Alien Campfire'
    else:
        return 'N/A'
df_2013['Cause'] = df_2013['Cause #'].apply(map_cause_to_meaning)
df_2013.head()

,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,2013,8/17/2013,10/24/2013,4,256175.5,Campfire
1,2013,9/10/2013,9/16/2013,1,0.07944366,Lightning
2,2013,5/27/2013,5/28/2013,10,1.235317,Vehicle
3,2013,9/18/2013,9/19/2013,14,3.633029,Unknown / Unidentified
4,2013,7/8/2013,7/8/2013,2,0.608927,Equipment Use


In [6]:
df_2013.to_csv("outputs/fires_2013.csv",
                  encoding="utf-8", index=False, header=True)

In [13]:
# Attempt to convert 'Alarm Date' to datetime, coercing errors
df_2013['Alarm Date'] = pd.to_datetime(df_2013['Alarm Date'], errors='coerce')

# Display rows with invalid dates
invalid_dates = df_2013[df_2013['Alarm Date'].isna()]
print(invalid_dates)

# Drop rows where 'Alarm Date' is in 2017
df_2013 = df_2013[df_2013['Alarm Date'].dt.year != 2001]

# Continue with the processing
df_byalarm = df_2013.sort_values(by='Alarm Date').reset_index()
df_byalarm.head()

Empty DataFrame
Columns: [Year, Alarm Date, Containment Date, Cause #, GIS Calculated Acres, Cause]
Index: []


,index,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,95,2013,2013-01-22,2013-01-24,5,318.5916,Debris
1,55,2013,2013-01-26,2013-05-26,9,10.54301,Miscellaneous
2,294,2013,2013-01-27,2013-01-27,14,13.56809,Unknown / Unidentified
3,293,2013,2013-02-24,2013-03-01,14,406.8415,Unknown / Unidentified
4,188,2013,2013-02-28,2013-03-02,14,362.2378,Unknown / Unidentified


In [14]:
earliest_fire = df_byalarm['Alarm Date'][0]
earliest_fire

Timestamp('2013-01-22 00:00:00')

In [15]:
# Convert 'Containment Date' to datetime, coercing errors
df_2013['Containment Date'] = pd.to_datetime(df_2013['Containment Date'], errors='coerce')

# Drop rows with invalid 'Containment Date'
df_2013_filtered = df_2013.dropna(subset=['Containment Date'])

# Now you can sort and process the DataFrame as needed
df_bycontainment = df_2013_filtered.sort_values(by='Containment Date', ascending=False).reset_index(drop=True)
latest_containment = df_bycontainment['Containment Date'].iloc[0]

df_bycontainment.head()

,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,2013,2013-12-29,2013-12-29,14,12.89286,Unknown / Unidentified
1,2013,2013-12-16,2013-12-20,9,913.963,Miscellaneous
2,2013,2013-05-30,2013-12-18,9,30267.69,Miscellaneous
3,2013,2013-12-18,2013-12-18,14,19.82545,Unknown / Unidentified
4,2013,2013-12-10,2013-12-10,11,44.73134,Powerline


In [16]:
latest_containment

Timestamp('2013-12-29 00:00:00')

In [17]:
output_path = os.path.join("outputs", "timeline_dates_2013.csv")
with open(output_path, 'w') as csvfile:

    # Initialize csv.writer
    csvwriter = csv.writer(csvfile, delimiter=',')

    # Write the first row (column headers)
    csvwriter.writerow(['Title', 'Date'])
    csvwriter.writerow(['Earliest Fire', earliest_fire])

    # Write the second row
    csvwriter.writerow(['Latest Containment', latest_containment])